<a href="https://colab.research.google.com/github/sbaskey/Deep-Learning/blob/master/mnst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing necessary library
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras import optimizers
from keras.callbacks import ModelCheckpoint,EarlyStopping
import time
from keras.layers import LeakyReLU
from tensorflow.keras import layers

Using TensorFlow backend.


In [0]:
batch_size = 128
num_classes = 10
epochs = 20

In [3]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(gx_train, gy_train), (gx_test, gy_test) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [0]:
if K.image_data_format() == 'channels_first':
    gx_train = gx_train.reshape(gx_train.shape[0], 1, img_rows, img_cols)
    gx_test = gx_test.reshape(gx_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    gx_train = gx_train.reshape(gx_train.shape[0], img_rows, img_cols, 1)
    gx_test = gx_test.reshape(gx_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [5]:
print('x_train shape:', gx_train.shape)
print(gx_train.shape[0], 'train samples')
print(gx_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [0]:
#partitioning validation size as 5000
gx_val, gx_test, gy_val, gy_test = train_test_split(gx_test, gy_test, test_size=0.5)

In [0]:
#one hot encoding 
gy_train = keras.utils.to_categorical(gy_train, num_classes)
gy_val = keras.utils.to_categorical(gy_val, num_classes)
gy_test = keras.utils.to_categorical(gy_test, num_classes)

In [8]:
gy_test[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32)

In [0]:
#function to train model with diffrenrent hyper parameter 
def do_mnst(model_,lr,nor,epochs):
  if nor==1:
    global gx_train
    global gy_train
    global gx_test
    global gx_val
    global gy_test
    global gy_val
    x_train=gx_train.astype('float32')
    x_test=gx_test.astype('float32')
    x_val=gx_val.astype('float32')
    y_train=gy_train.astype('float32')
    y_test=gy_test.astype('float32')
    y_val=gy_val.astype('float32')

    x_train /= 255.0
    x_test /= 255.0
    x_val/=255.0
    y_test/=255.0
    y_val/=255.0
    y_test/=255.0
    #x_train.shape
  else:
     x_train=gx_train
     x_test=gx_test
     x_val=gx_val
     y_train=gy_train
     y_test=gy_test
     y_val=gy_val


  model_.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=optimizers.SGD(lr=lr),
              metrics=['accuracy'])
  t1=time.time()
  model_.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=0,
          validation_data=(x_val, y_val))
  t2=time.time()
  score = model_.evaluate(x_test, y_test, verbose=0)
  print('Test loss:', score[0])
  print('Test accuracy:', score[1])
 # print(model_.summary())
  print("time_takent to train the model",t2-t1)


model1 with only 1 hidden layer

In [10]:
#model with only one hidden layer
model1 = Sequential()
model1.add(Flatten())
model1.add(Dense(32, activation='sigmoid'))
model1.add(Dropout(0.5))
model1.add(Dense(num_classes, activation='softmax'))

In [11]:
print("without normalization")
do_mnst(model1,0.1,0,20)

without normalization




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Test loss: 0.5269745308876037
Test accuracy: 0.8524
time_takent to train the model 20.280267477035522


In [12]:

print("with normalization")
do_mnst(model1,0.1,1,20)

with normalization
Test loss: 4.190840908631799e-06
Test accuracy: 0.9208
time_takent to train the model 20.515926837921143


Accuracy gets better with normalization

In [0]:
#model 2 with 2 hidden layer
model2 = Sequential()
model2.add(Flatten())
model2.add(Dense(32, activation='sigmoid'))
model2.add(Dense(32, activation='sigmoid'))
model2.add(Dropout(0.5))
model2.add(Dense(num_classes, activation='softmax'))

In [0]:
#model 3 with 3hidden layer
model3 = Sequential()
model3.add(Flatten())
model3.add(Dense(32, activation='sigmoid'))
model3.add(Dense(32, activation='sigmoid'))
model3.add(Dense(32, activation='sigmoid'))
model3.add(Dropout(0.5))
model3.add(Dense(num_classes, activation='softmax'))

In [15]:
print("with 2 hiden layer")
do_mnst(model2,0.1,1,20)

with 2 hiden layer
Test loss: 4.250770599173847e-06
Test accuracy: 0.92
time_takent to train the model 20.348302125930786


In [16]:
#with 3 hidden layer
do_mnst(model3,0.1,1,20)

Test loss: 6.243836991779972e-06
Test accuracy: 0.8988
time_takent to train the model 21.10459566116333


Model (2 hiden layer is doing better in terms of timing and accuracy)

In [17]:
#model 2 with lr=0.001
do_mnst(model2,0.001,0,20)

Test loss: 0.28711489325761796
Test accuracy: 0.9184
time_takent to train the model 20.22449254989624


In [18]:
#model 2 with lr=0.0001
do_mnst(model1,0.0001,0,20)

Test loss: 0.3030007659673691
Test accuracy: 0.9088
time_takent to train the model 19.69685173034668


  model2 with lr=0.001 is giving more accuracy compared to lr=0.0001

In [0]:
#model 2 with 2 hidden layer and nodes 64
model2 = Sequential()
model2.add(Flatten())
model2.add(Dense(64, activation='sigmoid'))
model2.add(Dense(64, activation='sigmoid'))
model2.add(Dropout(0.5))
model2.add(Dense(num_classes, activation='softmax'))

In [20]:
#model 2 with node 64
do_mnst(model2,0.001,0,20)

Test loss: 1.5944037229537964
Test accuracy: 0.756
time_takent to train the model 25.183111429214478


In [0]:
#model 2 with 2 hidden layer and nodes 128
model2 = Sequential()
model2.add(Flatten())
model2.add(Dense(128, activation='sigmoid'))
model2.add(Dense(128, activation='sigmoid'))
model2.add(Dropout(0.5))
model2.add(Dense(num_classes, activation='softmax'))

In [22]:
#model 2 with node 128 and lr=0.001
do_mnst(model2,0.001,0,20)

Test loss: 1.1983607822418212
Test accuracy: 0.7962
time_takent to train the model 37.21574354171753


          model with 2 hidden 32 nodes is doing better than 64 and 128 nodes

In [0]:
#model with 2 hidden layer and nodes 32  and relu activation function
model3 = Sequential()
model3.add(Flatten())
model3.add(Dense(32, activation='relu'))
model3.add(Dense(32, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(num_classes, activation='softmax'))

In [0]:
#model with 2 hidden layer and nodes 32 and tanh  activation function
model4 = Sequential()
model4.add(Flatten())
model4.add(Dense(32, activation='tanh'))
model4.add(Dense(32, activation='tanh'))
model4.add(Dropout(0.5))
model4.add(Dense(num_classes, activation='softmax'))

In [25]:
"""#model with 2 hidden layer and nodes 32 and leaky_relu activation function
model5 = Sequential()
model5.add(Flatten())
model5.add(Dense(32, activation='leaky_relu'))
model5.add(Dense(32, activation='leaky_relu'))
model5.add(Dropout(0.5))
model5.add(Dense(num_classes, activation='softmax'))"""

"#model with 2 hidden layer and nodes 32 and leaky_relu activation function\nmodel5 = Sequential()\nmodel5.add(Flatten())\nmodel5.add(Dense(32, activation='leaky_relu'))\nmodel5.add(Dense(32, activation='leaky_relu'))\nmodel5.add(Dropout(0.5))\nmodel5.add(Dense(num_classes, activation='softmax'))"

In [26]:
do_mnst(model3,0.001,0,20)

Test loss: 0.7109663108825683
Test accuracy: 0.8
time_takent to train the model 21.22269082069397


In [27]:
do_mnst(model4,0.001,0,20)

Test loss: 0.4666232788085938
Test accuracy: 0.8864
time_takent to train the model 20.76862359046936


    sigmoid activation function with lr=0.001 and 32 nodes with 2 layer is     giving maximum score among all the models in terms of timing and accuracy
    